In [2]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 30.2 MB/s eta 0:00:00


**Decision variables**: For each potential station location, a binary variable (0 or 1) can be used to indicate whether or not a station is placed at that location.

**Objective function**: Minimize the total cost of building and maintaining the stations.

**Constraints**: 

1.   The total number of stations that can be built is limited by budget and city regulations.

2.   The minimum and maximum number of bikes that must be available at each station.

3. The maximum distance people are willing to walk to reach a station.

4. The demand for bikes in different areas of the city.


In [ ]:
from pulp import *

# Define the data
num_locations = 10
cost = [1000, 800, 600, 700, 900, 800, 500, 600, 900, 1000]
demand = [50, 40, 30, 20, 30, 20, 10, 20, 30, 40]
distance = [1, 2, 3, 4, 5, 2, 1, 3, 4, 2]
max_stations = 5
min_bikes = 200
max_distance = 10

# Define the decision variables
x = [LpVariable(f"x{i}", cat='Binary') for i in range(num_locations)]

# Define the objective function
prob = LpProblem("Bicycle Station Location", LpMinimize)
prob += lpSum([cost[i]*x[i] for i in range(num_locations)])

# Define the constraints
prob += lpSum([x[i] for i in range(num_locations)]) <= max_stations
prob += lpSum([demand[i]*x[i] for i in range(num_locations)]) >= min_bikes
prob += lpSum([distance[i]*x[i] for i in range(num_locations)]) <= max_distance

# Solve the LP model
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
for i in range(num_locations):
    if x[i].value() == 1:
        print(f"A station should be built at location {i}")


**Decision variables**: The x and y coordinates of each station

**Objective function**: Minimize the total distance between the stations and the demand points. This could be represented as:

minimize $(Σ(sqrt((x_i - d_x)^2 + (y_i - d_y)^2)) for all i)$

where x_i and y_i are the coordinates of station i, and d_x and d_y are the coordinates of the demand point.


**Constraints**: 


1.   The total cost of installing the stations must be less than or equal to the allocated budget


In [ ]:
import random
from gurobipy import *

# Number of stations and demand points
num_stations = 5
num_demand_points = 10

# Maximum x and y coordinates of the space available for the stations
max_x = 100
max_y = 100

# Coordinates of the demand points
demand_x = [random.randint(0, max_x) for _ in range(num_demand_points)]
demand_y = [random.randint(0, max_y) for _ in range(num_demand_points)]

# Cost of each station
cost = [random.randint(50, 100) for _ in range(num_stations)]

# Budget for the project
budget = sum(cost) / 2

# Max and min x,y coordinates for each station
max_x_zone = [random.randint(0, max_x) for _ in range(num_stations)]
max_y_zone = [random.randint(0, max_y) for _ in range(num_stations)]
min_x_zone = [random.randint(0, max_x) for _ in range(num_stations)]
min_y_zone = [random.randint(0, max_y) for _ in range(num_stations)]

# Create the model
m = Model()

# Create the decision variables
x = {}
y = {}

for i in range(num_stations):
    x[i] = m.addVar(lb=min_x_zone[i], ub=max_x_zone[i], name="x_%d" % i)
    y[i] = m.addVar(lb=min_y_zone[i], ub=max_y_zone[i], name="y_%d" % i)

# Define the objective function
obj = quicksum(quicksum(sqrt((x[i] - demand_x[j])**2 + (y[i] - demand_y[j])**2) for j in range(num_demand_points)) for i in range(num_stations))
m.setObjective(obj, GRB.MINIMIZE)

# Define the constraints

# Budget constraint
m.addConstr(quicksum(cost[i] for i in range(num_stations)) <= budget)

# Optimize the model
m.optimize()

# Extract the solution
for i in range(num_stations):
    print("Station", i, ": (", x[i].x, ",", y[i].x, ")")


**Decision variables**: La cantidad de estaciones de cada tamaño que se instalen en cada ubicación.

**Objective function**: minimize (Costo total de instalación de estaciones) = Σ (Costo de cada tamaño de estación * Número de estaciones de ese tamaño en una ubicación)

**Restricciones**: 


1.   Satisfacer la demanda: Σ (Número de bicicletas en cada tamaño de estación * Número de estaciones de ese tamaño en una ubicación) >= Demanda total
2.   Distancia maxima entre estaciones 
3.   Número de estaciones de cada tamaño en una ubicación sea entero
4.   Número de estaciones de cada tamaño >= 0.


In [ ]:
from pulp import *

# Datos de entrada
costos = [10, 20, 30] # Costos de cada tamaño de estación
demand = [30, 20, 40, 10, 50] # Demanda en cada ubicación
ubicaciones = 5 # Número de ubicaciones
tamanos = 3 # Número de tamaños de estaciones
distancia_maxima = 10 # Distancia máxima entre estaciones

# Inicializar modelo
m = LpProblem("Bicicletas compartidas", LpMinimize)

# Variables de decisión
abrir = LpVariable.dicts("Abrir", [(i, j) for i in range(ubicaciones) for j in range(tamanos)], 0, 1, LpBinary)
estaciones = LpVariable.dicts("Estacion", [(i, j) for i in range(ubicaciones) for j in range(tamanos)], 0, None, LpInteger)

# Función objetivo
m += lpSum(costos[j] * abrir[i,j] for i in range(ubicaciones) for j in range(tamanos))

# Restricciones
for i in range(ubicaciones):
    m += lpSum(estaciones[i,j] for j in range(tamanos)) >= demand[i], "Demanda_"+str(i+1)
    for j in range(tamanos):
        m += estaciones[i,j] <= abrir[i,j] * 10000, "Estaciones_abiertas_"+str(i+1)+"_"+str(j+1)

for i in range(ubicaciones):
    for j in range(i+1, ubicaciones):
        m += lpSum(abrir[i,k] + abrir[j,k] for k in range(tamanos)) <= 1, "Distancia_maxima_"+str(i+1)+"_"+str(j+1)

# Resolver modelo
m.solve()

# Imprimir resultados
for i in range(ubicaciones):
    for j in range(tamanos):
        if abrir[i,j].value() > 0:
            print("Ubicación", i+1, "tamaño", j+1, "cantidad", estaciones[i,j].value())


# **Codigo ganador**

**Decision variables**: Binary variables indicating if a station is opened and what size it is
$x_{i,j}$ donde $i ∈ I$: set of stations y $j \in J$: set of sizes

**Objective function**: minimize costs
Min $∑x_{i,j}$*$c_{j}$   sum of costs for each size

**Restricciones**: 


1.   Cover the demand at each location
$∑i$ $∑j$ $x_{i,j}$ = $d_i$ 
2.   Maximum distance between stations
$x_{i,j}$*$x_{l,j}$*distance$_{i,l}$ <= max_distance


In [4]:
import numpy as np
import pulp

# Invented data for 5 potential locations in Bogota
location_names = ['Location 1', 'Location 2', 'Location 3', 'Location 4', 'Location 5']
demand = np.random.randint(100, 1000, size=5)
distance = np.random.randint(1, 10, size=5)

# Costs for small, medium, and large stations
small_cost = 1000
medium_cost = 2000
large_cost = 3000

# Maximum distance between stations
max_distance = 7

# Define the decision variables
x = pulp.LpVariable.dicts('x', [(i, j) for i in range(len(location_names)) for j in range(3)],
                           cat=pulp.LpBinary) # binary variables indicating if a station is opened and what size it is

# Define the problem
prob = pulp.LpProblem('Bicycle Station Optimization', pulp.LpMinimize)

# Define the constraints
prob += pulp.lpSum([x[(i, j)] for i in range(len(location_names)) for j in range(3)]) == demand[i], f'Demand {i}'
prob += pulp.lpSum([x[(i, j)] * distance[i] for i in range(len(location_names)) for j in range(3)]) <= max_distance, 'Max Distance'
for i in range(len(location_names)):
    prob += pulp.lpSum([x[(i, j)] for j in range(3)]) <= 1, f'Size {i}'

# Define the objective function
prob += small_cost * pulp.lpSum([x[(i, 0)] for i in range(len(location_names))]) + \
        medium_cost * pulp.lpSum([x[(i, 1)] for i in range(len(location_names))]) + \
        large_cost * pulp.lpSum([x[(i, 2)] for i in range(len(location_names))])

# Solve the problem
prob.solve()

# Print the results
print(f'Optimal Cost: {pulp.value(prob.objective)}')
for i, location in enumerate(location_names):
    if x[(i, 0)].varValue:
        print(f'{location}: Small Station')
    elif x[(i, 1)].varValue:
        print(f'{location}: Medium Station')
    elif x[(i, 2)].varValue:
        print(f'{location}: Large Station')

Optimal Cost: 4147000.0
Location 1: Small Station
Location 2: Small Station
Location 3: Small Station
Location 4: Small Station
Location 5: Small Station


# **Intento de unir los problemas**

Esta es la misma definicion que el anterior solo que se le añade la parte de predecir una nueva demanda y evaluar el rebalanceo 

In [ ]:
import numpy as np
import pulp

# Invented data for 5 potential locations in Bogota
location_names = ['Location 1', 'Location 2', 'Location 3', 'Location 4', 'Location 5']
demand = np.random.randint(100, 1000, size=5)
distance = np.random.randint(1, 10, size=5)

# Costs for small, medium, and large stations
small_cost = 1000
medium_cost = 2000
large_cost = 3000

# Maximum distance between stations
max_distance = 7

# Define the decision variables
x = pulp.LpVariable.dicts('x', [(i, j) for i in range(len(location_names)) for j in range(3)],
                           cat=pulp.LpBinary) # binary variables indicating if a station is opened and what size it is

# Define the problem
prob = pulp.LpProblem('Bicycle Station Optimization', pulp.LpMinimize)

# Define the constraints
prob += pulp.lpSum([x[(i, j)] for i in range(len(location_names)) for j in range(3)]) == demand[i], f'Demand {i}'
prob += pulp.lpSum([x[(i, j)] * distance[i] for i in range(len(location_names)) for j in range(3)]) <= max_distance, 'Max Distance'
for i in range(len(location_names)):
    prob += pulp.lpSum([x[(i, j)] for j in range(3)]) <= 1, f'Size {i}'

# Define the objective function
prob += small_cost * pulp.lpSum([x[(i, 0)] for i in range(len(location_names))]) + \
        medium_cost * pulp.lpSum([x[(i, 1)] for i in range(len(location_names))]) + \
        large_cost * pulp.lpSum([x[(i, 2)] for i in range(len(location_names))])

# Solve the problem
prob.solve()

# Store the results
optimal_size = [0] * len(location_names)
for i, location in enumerate(location_names):
    if x[(i, 0)].varValue:
        optimal_size[i] = 'Small'
    elif x[(i, 1)].varValue:
        optimal_size[i] = 'Medium'
    elif x[(i, 2)].varValue:
        optimal_size[i] = 'Large'

# Forecast the new demand
new_demand = demand + np.random.randint(-100, 100, size=5)

# Evaluate the need for rebalancing
rebalance = [0] * len(location_names)
for i, location in enumerate(location_names):
    if new_demand[i] > demand[i] and optimal_size[i] == 'Small':
        rebalance[i] = 'Upgrade to Medium'
    elif new_demand[i] > demand[i] and optimal_size[i] == 'Medium':
        rebalance[i] = 'Upgrade to Large'
    elif new_demand[i] < demand[i] and optimal_size[i] == 'Large':
        rebalance[i] = 'Downgrade to Medium'
    elif new_demand[i] < demand[i] and optimal_size[i] == 'Medium':
        rebalance[i] = 'Downgrade to Small'
    else:
        rebalance[i] = 'No Change'

print('Optimal Station Sizes: ', optimal_size)
print('New Demand: ', new_demand)
print('Rebalance Recommendations: ', rebalance)
     